Transformer-based NLP topic modeling using the Python package BERTopic: modeling, prediction, and visualization

# Resources

- [Blog post](https://medium.com/grabngoinfo/topic-modeling-with-deep-learning-using-python-bertopic-cf91f5676504) for this tutorial
- Video version of the tutorial on [YouTube](https://www.youtube.com/watch?v=Q5g0OnoyC5U&list=PLVppujud2yJpx5r8GFeJ81fyek8dEDMX-&index=4)
- More video tutorials on [NLP](https://www.youtube.com/playlist?list=PLVppujud2yJpx5r8GFeJ81fyek8dEDMX-)
- More blog posts on [NLP](https://medium.com/@AmyGrabNGoInfo/list/nlp-49340193610f)


For more information about data science and machine learning, please check out my [YouTube channel](https://www.youtube.com/channel/UCmbA7XB6Wb7bLwJw9ARPcYg), [Medium Page](https://medium.com/@AmyGrabNGoInfo) and [GrabNGoInfo.com](https://grabngoinfo.com/tutorials/), or follow GrabNGoInfo on [LinkedIn](https://www.linkedin.com/company/grabngoinfo/).

# Intro

BERTopic is a topic modeling python library that uses the combination of transformer embeddings and clustering model algorithms to identify topics in NLP (Natual Language Processing). In this tutorial, we will talk about:
* How transformers, c-TF-IDF, and clustering models are used behind the BERTopic?
* How to extract and interpret topics from the topic modeling results?
* How to make predictions using topic modeling?
* How to save and load a BERTopic topic model?

This is an introduction to the BERTopic model. To learn how to optimize the BERTopic model, please check out [Hyperparameter Tuning for BERTopic Model in Python](https://medium.com/grabngoinfo/hyperparameter-tuning-for-bertopic-model-in-python-104445778347).

Let's get started!

# Step 0: BERTopic Model Algorithms

In step 0, we will talk about the algorithms behind the BERTopic model.
* **Documents Embedding**: Firstly, we need to get the embeddings for all the documents. Embeddings are the vector representation of the documents.
 * BERTopic uses the English version of the `sentence_transformers` by default to get document embeddings.
 * If there are multiple languages in the document, we can use `BERTopic(language="multilingual")` to support the topic modeling of over 50 languages.
 * BERTopic also supports the pre-trained models from other python packages such as hugging face and flair.
* **Documents Clustering**: After the text documents have been transformed into embeddings, the next step is to run a clustering model on the embedded documents. Because the embedding vectors usually have very high dimensions, dimension reduction techniques are used to reduce the dimensionalities.
 * The default algorithm for dimension reduction is UMAP (Uniform Manifold Approximation & Projection). Compared with other dimension reduction techniques such as PCA (Principle Component Analysis), UMAP maintains the data's local and global structure when reducing the dimensionality, which is important for representing the semantics of the text data. BERTopic provides the option of using other dimensionality reduction techniques by changing the `umap_model` value in the `BERTopic` method.
 * The default algorithm for clustering is HDBSCAN. HDBSCAN is a density-based clustering model. It identifies the number of clustering automatically, and does not require specifying the number of clusters beforehand like most of the clustering models.
* **Topic Representation**: After assigning each document in the corpus into a cluster, the next step is to get the topic representation using a class-based TF-IDF called c-TF-IDF. The top words with the highest c-TF-IDF scores are selected to represent each topic.
 * c-TF-IDF is similar to TF-IDF in that it measures the term importance by term frequencies while taking into account the whole corpus (all the text data for the analysis).
 * c-TF-IDF is different from TF-IDF in that the term frequency level is different. In the regular TF-IDF, TF measures the term frequency in each document. While in the c-TF-IDF, TF measures the term frequency in each cluster, and each cluster includes many documents.
* **Maximal Marginal Relevance (MMR)** (optional): After extracting the most important terms describing each cluster, there is an optional step to optimize the terms using Maximal Marginal Relevance (MMR). Maximal Marginal Relevance (MMR) has two benefits:
 * The first benefit is to increase the coherence among the terms for the same topic and remove irrelevant terms.
 * The second benefit is to increase the topic representation by removing synonyms and variations of the same words.


# Step 1: Install And Import Python Libraries

In step 1, we will install and import python libraries.

Firstly, let's import `bertopic`.

In [1]:
# Install bertopic
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 54.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
    

After installing `bertopic`, when we tried to import the `BERTopic` method, a type error about an unexpected keyword argument `cachedir` came up.

This `TypeError` is caused by the incompatibility between `joblib` and `HDBSCAN`. At the time this tutorial was created, `joblib` has a new release that is not supported by `HDBSCAN`. HDBSCAN does have a fix for it but has not been rolled out. So if you are watching this tutorial on YouTube or reading this tutorial on Medium.com at a later time, you may not encounter this error message.

In [2]:
# Try to import BERTopic
from bertopic import BERTopic

Before the incompatibility issue between `joblib` and `HDBSCAN` is fixed, we can solve this issue by installing an old version of `joblib`. In this example, we used `joblib` version 1.1.0. After installing `joblib`, we need to restart the runtime.

In [ ]:
# Install older version of joblib
!pip install --upgrade joblib==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 306 kB 5.0 MB/s 
  Attempting uninstall: joblib
    Found existing installation: joblib 1.2.0
    Uninstalling joblib-1.2.0:
      Successfully uninstalled joblib-1.2.0


After installing the python packages, we will import the python libraries.
* `pandas` and `numpy` are imported for data processing.
* `nltk` is imported for text preprocessing. We downloaded the information for removing stopwords and lemmatization from `nltk`.
* `BERTopic` is imported for the topic modeling.
* `UMAP` is for dimension reduction.


In [3]:
# Data processing
import pandas as pd
import numpy as np

# Text preprocessiong
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()

# Topic model
from bertopic import BERTopic

# Dimension reduction
from umap import UMAP

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


# Step 2: Download And Read Data

The second step is to download and read the dataset.

The UCI Machine Learning Repository has the review data from three websites: imdb.com, amazon.com, and yelp.com. We will use the review data from amazon.com for this tutorial. Please follow these steps to download the data.
1. Go to: https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences
2. Click "Data Folder"
3. Download "sentiment labeled sentences.zip"
4. Unzip "sentiment labeled sentences.zip"
5. Copy the file "amazon_cells_labelled.txt" to your project folder

Those who are using Google Colab for this analysis need to mount Google Drive to read the dataset. You can ignore the code below if you are not using Google Colab.
* `drive.mount` is used to mount to the Google drive so the colab notebook can access the data on the Google drive.
* `os.chdir` is used to change the default directory on Google drive. I set the default directory to the folder where the review dataset is saved.
* `!pwd` is used to print the current working directory.

Please check out [Google Colab Tutorial for Beginners](https://medium.com/towards-artificial-intelligence/google-colab-tutorial-for-beginners-834595494d44) for details about using Google Colab for data science projects.

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Change directory
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

# Print out the current directory
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


Now let's read the data into a `pandas` dataframe and see what the dataset looks like.

The dataset has two columns. One column contains the reviews and the other column contains the sentiment label for the review. Since this tutorial is for topic modeling, we will not use the sentiment label column, so we removed it from the dataset.

In [5]:
# Read in data
amz_review = pd.read_csv('amazon_cells_labelled.txt', sep='\t', names=['review', 'label'])

# Drop te label
amz_review = amz_review.drop('label', axis=1);

# Take a look at the data
amz_review.head()

,review
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


`.info` helps us to get information about the dataset.

From the output, we can see that this data set has 1000 records, and no missing data. The 'review' column is the `object` type.

In [6]:
# Get the dataset information
amz_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1000 non-null   object
dtypes: object(1)
memory usage: 7.9+ KB


# Step 3: Text Data Preprocessing (Optional)

In step 3, we included some sample code for text data preprocessing.

Generally speaking, there is no need to preprocess the text data when using the python BERTopic model. However, since our dataset is a simple dataset, a lot of stopwords are picked to represent the topics.

Therefore, we removed stopwords and did lemmatization as data preprocessing. But please ignore this step if this is not an issue for you.

In [7]:
# Remove stopwords
stopwords = nltk.corpus.stopwords.words('english')
print(f'There are {len(stopwords)} default stopwords. They are {stopwords}')

There are 179 default stopwords. They are ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'no

There are 179 default stopwords in the `nltk` library. We printed all the stopwords out to see what words are considered to be stopwords. `nltk` provides the option to add customized stopwords to the list.



Lemmatization refers to changing words to their base form.

After removing stopwords and lemmatizing the words we can see that the stopwords like `to` and `for` are removed, and the word like `conversations` is converted to `conversation`.

In [8]:
# Remove stopwords
amz_review['review_without_stopwords'] = amz_review['review'].apply(lambda x: ' '.join([w for w in x.split() if w.lower() not in stopwords]))

# Lemmatization
amz_review['review_lemmatized'] = amz_review['review_without_stopwords'].apply(lambda x: ' '.join([wn.lemmatize(w) for w in x.split() if w not in stopwords]))

# Take a look at the data
amz_review.head()

,review,review_without_stopwords,review_lemmatized
0,So there is no way for me to plug it in here i...,way plug US unless go converter.,way plug US unless go converter.
1,"Good case, Excellent value.","Good case, Excellent value.","Good case, Excellent value."
2,Great for the jawbone.,Great jawbone.,Great jawbone.
3,Tied to charger for conversations lasting more...,Tied charger conversations lasting 45 minutes....,Tied charger conversation lasting 45 minutes.M...
4,The mic is great.,mic great.,mic great.


# Step 4: Topic Modeling Using BERTopic

In step 4, we will build the topic model using BERTopic.

BERTopic model by default produces different results each time because of the stochasticity inherited from UMAP.

To get reproducible topics, we need to pass a value to the `random_state` parameter in the `UMAP` method.
* `n_neighbors=15` means that the local neighborhood size for UMAP is 15. This is the parameter that controls the local versus global structure in data.
 * A low value forces UMAP to focus more on local structure, and may lose insights into the big picture.
 * A high value pushes UMAP to look at broader neighborhood, and may lose details on local structure.
 * The default `n_neighbors` values for UMAP is 15.
* `n_components=5` indicates that the target dimension from UMAP is 5. This is the dimension of data that will be passed into the clustering model.
* `min_dist` controls how tightly UMAP is allowed to pack points together. It's the minimum distance between points in the low dimensional space.
 * Small values of `min_dist` result in clumpier embeddings, which is good for clustering. Since our goal of dimension reduction is to build clustering models, we set `min_dist` to 0.
 * Large values of `min_dist` prevent UMAP from packing points together and preserves the broad structure of data.
* `metric='cosine'` indicates that we will use cosine to measure the distance.
* `random_state` sets a random seed to make the UMAP results reproducible.

After initiating the UMAP model, we pass it to the BERTopic model, set the language to be English, and set the `calculate_probabilities` parameter to `True`.

Finally, we pass the processed review documents to the topic model and saved the results for topics and topic probabilities.
 * The values in `topics` represents the topic each document is assigned to.
 * The values in `probabilities` represents the probability of a document belongs to each of the topics.

In [9]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True)

# Run BERTopic model
topics, probabilities = topic_model.fit_transform(amz_review['review_lemmatized'])

# Step 5: Extract Topics From Topic Modeling

In step 5, we will extract topics from the BERTopic modeling results.

Using the attribute `get_topic_info()` on the topic model gives us the list of topics. We can see that the output gives us 31 rows in total.
* Topic -1 should be ignored. It indicates that the reviews are not assigned to any specific topic. The count for topic -1 is 322, meaning that there are 322 reviews as outliers that do not belong to any topic.
* Topic 0 to topic 29 are the 30 topics created for the reviews. It was ordered by the number of reviews in each topic, so topic 0 has the highest number of reviews.
* The `Name` column lists the top terms for each topic. For example, the top 4 terms for Topic 0 are sound, quality, volume, and audio, indicating that it is a topic related to sound quality.

In [10]:
# Get the list of topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,327,-1_phone_service_it_problem,"[phone, service, it, problem, work, make, call...","[Worst Customer Service Ever., phone work grea..."
1,0,65,0_sound_quality_volume_audio,"[sound, quality, volume, audio, poor, low, hea...","[Excellent sound quality., sound quality great..."
2,1,56,1_headset_bluetooth_headphone_best,"[headset, bluetooth, headphone, best, bt, soun...","[Love headset!, Excellent bluetooth headset., ..."
3,2,45,2_ear_earpiece_comfortable_fit,"[ear, earpiece, comfortable, fit, jabra, comfo...","[Jabra EarGels fit ear well., I've tried sever..."
4,3,35,3_battery_life_original_dying,"[battery, life, original, dying, last, long, r...","[Battery life also great!, battery work great!..."
5,4,34,4_price_value_good_deal,"[price, value, good, deal, bargain, product, a...","[Great product price!., Great product price., ..."
6,5,32,5_disappointed_disappointment_decision_mistake,"[disappointed, disappointment, decision, mista...","[disappointed., Disappointed!., disappointed.]"
7,6,30,6_charger_plug_car_work,"[charger, plug, car, work, charging, current, ...",[got car charger even week charger broken...I ...
8,7,27,7_case_holster_sex_fit,"[case, holster, sex, fit, well, made, seems, p...",[little geeky think thats sex toast rock ooze ...
9,8,26,8_phone_great_extremely_gosh,"[phone, great, extremely, gosh, exclaim, tv, e...","[Great phone!., great phone!., Great Phone.]"


If more than 4 terms are needed for a topic, we can use `get_topic` and pass in the topic number. For example, `get_topic(0)` gives us the top 10 terms for topic 0 and their relative importance.

In [11]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('sound', 0.11468165718597091),
 ('quality', 0.07866845435936802),
 ('volume', 0.06365637372489294),
 ('audio', 0.05028365001733581),
 ('poor', 0.045802667097527436),
 ('low', 0.038833194358717134),
 ('hear', 0.035364652069384966),
 ('clear', 0.035364652069384966),
 ('loud', 0.03504068576005251),
 ('end', 0.033522433344890544)]

We can visualize the top keywords using a bar chart. `top_n_topics=12` means that we will create bar charts for the top 12 topics. The length of the bar represents the score of the keyword. A longer bar means higher importance for the topic.

In [12]:
# Visualize top topic keywords
topic_model.visualize_barchart(top_n_topics=12)

Another view for keyword importance is the "Term score decline per topic" chart. It's a line chart with the term rank being the x-axis and the c-TF-IDF score on the y-axis.

There are a total of 31 lines, one line for each topic. Hovering over the line shows the term score information.

In [13]:
# Visualize term rank decrease
topic_model.visualize_term_rank()

# Step 6: Topic Similarities

In step 6, we will analyze the relationship between the topics generated by the topic model.

We will use three visualizations to study how the topics are related to each other. The three methods are intertopic distance map, the hierarchical clustering of topics, and the topic similarity matrix.

Intertopic distance map measures the distance between topics. Similar topics are closer to each other, and very different topics are far from each other. From the visualization, we can see that there are five topic groups for all the topics. Topics with similar semantic meanings are in the same topic group.

The size of the circle represents the number of documents in the topics, and larger circles mean that more reviews belong to the topic.

In [14]:
# Visualize intertopic distance
topic_model.visualize_topics()

Another way to see how the topics are connected is through a hierarchical clustering graph. We can control the number of topics in the graph by the `top_n_topics` parameter.

In this example, the top 10 topics are included in the hierarchical graph. We can see that the sound quality topic is closely connected to the headset topic, and both of them are connected to the earpiece comfortable topic.

In [15]:
# Visualize connections between topics using hierachical clustering
topic_model.visualize_hierarchy(top_n_topics=10)

Heatmap can also be used to analyze the similarities between topics. The similarity score ranges from 0 to 1. A value close to 1 represents a higher similarity between the two topics, which is represented by darker blue color.

In [16]:
# Visualize similarity using heatmap
topic_model.visualize_heatmap()

# Step 7: Topic Model Predicted Probabilities

In step 7, we will talk about how to use BERTopic model to get predicted probabilities.

The topic prediction for a document is based on the predicted probabilities of the document belonging to each topic. The topic with the highest probability is the predicted topic. This probability represents how confident we are about finding the topic in the document.

We can visualize the probabilities using `visualize_distribution`, and pass in the document index. `visualize_distribution` has the default probability threshold of 0.015, so only the topic with a probability greater than 0.015 will be included.

In [17]:
# Visualize probability distribution
topic_model.visualize_distribution(topic_model.probabilities_[0], min_probability=0.015)

If you would like to save the visualization as a separate html file, we can save the chart into a variable and use `write_html` to write the chart into a file.

In [18]:
# Save the chart to a variable
chart = topic_model.visualize_distribution(topic_model.probabilities_[0])

# Write the chart as a html file
chart.write_html("amz_review_topic_probability_distribution.html")

The topic probability distribution for the first review in the dataset shows that topic 7 has the highest probability, so topic 7 is the predicted topic.

The first review is "So there is no way for me to plug it in here in the US unless I go by a converter.", and the topic of plugging a charger is pretty relevant.

In [19]:
# Check the content for the first review
amz_review['review'][0]

'So there is no way for me to plug it in here in the US unless I go by a converter.'

We can also get the predicted probability for all topics using the code below.

In [20]:
# Get probabilities for all topics
topic_model.probabilities_[0]

array([0.01016367, 0.00816787, 0.00754525, 0.02579086, 0.00714906,
       0.00869511, 0.08848236, 0.00910779, 0.01021896, 0.00979041,
       0.01017699, 0.00706541, 0.01349993, 0.01148849, 0.00961576,
       0.00709405, 0.00728534, 0.00971712, 0.01642342, 0.00834035,
       0.00865261, 0.01082052, 0.01364636, 0.01149314, 0.01055254,
       0.01401247, 0.00979215, 0.00776336, 0.01122303])

We can see that there are 30 probability values, one for each topic. The index 7 has the highest value, indicating that topic 7 is the predicted topic.

# Step 8: Topic Model In-sample Predictions

In step 8, we will talk about how to make in-sample predictions using the topic model.

BERTopic model can output the predicted topic for each review in the dataset.

Using `.topics_`, we save the predicted topics in a list and then save it as a column in the review dataset.

In [21]:
# Get the topic predictions
topic_prediction = topic_model.topics_[:]

# Save the predictions in the dataframe
amz_review['topic_prediction'] = topic_prediction

# Take a look at the data
amz_review.head()

,review,review_without_stopwords,review_lemmatized,topic_prediction
0,So there is no way for me to plug it in here i...,way plug US unless go converter.,way plug US unless go converter.,6
1,"Good case, Excellent value.","Good case, Excellent value.","Good case, Excellent value.",4
2,Great for the jawbone.,Great jawbone.,Great jawbone.,-1
3,Tied to charger for conversations lasting more...,Tied charger conversations lasting 45 minutes....,Tied charger conversation lasting 45 minutes.M...,6
4,The mic is great.,mic great.,mic great.,0


# Step 9: Topic Model Predictions on New Data

In step 9, we will talk about how to use the BERTopic model to make predictions on new reviews.

Let's say there is a new review "I like the new headphone. Its sound quality is great.", and we would like to automatically predict the topic for this review.
* Firstly, let's decide the number of topics to include in the prediction.
 * If we would like to assign only one topic to the document, then the number of topics should be 1.
 * If we would like to assign multiple topics to the document, then the number of topics should be greater than 1. Here we are getting the top 3 topics that are most relevant to the new review.
* After that, we pass the new review and the number of topics to the `find_topics` method. This gives us the topic number and the similarity value.
* Finally, the results are printed. The top 3 similar topics for the new review are topic 1, topic 0, and topic 2. The similarities are 0.43, 0.34, and 0.30.


In [22]:
# New data for the review
new_review = "I like the new headphone. Its sound quality is great."

# Find topics
num_of_topics = 3
similar_topics, similarity = topic_model.find_topics(new_review, top_n=num_of_topics);

# Print results
print(f'The top {num_of_topics} similar topics are {similar_topics}, and the similarities are {np.round(similarity,2)}')

The top 3 similar topics are [1, 0, 28], and the similarities are [0.62 0.57 0.51]


To verify if the assigned topics are a good fit for the new review, let's get the top keywords for the top 3 topics using the `get_topic` method.

In [23]:
# Print the top keywords for the top similar topics
for i in range(num_of_topics):
  print(f'The top keywords for topic {similar_topics[i]} are:')
  print(topic_model.get_topic(similar_topics[i]))

The top keywords for topic 1 are:
[('headset', 0.1709460313360213), ('bluetooth', 0.05077700279133207), ('headphone', 0.037418838472997144), ('best', 0.03385044191366794), ('bt', 0.0319497968718133), ('sound', 0.030194441308607194), ('really', 0.026810616978861387), ('looking', 0.02553263057875083), ('ever', 0.024340578710446134), ('logitech', 0.023372329049310102)]
The top keywords for topic 0 are:
[('sound', 0.11468165718597091), ('quality', 0.07866845435936802), ('volume', 0.06365637372489294), ('audio', 0.05028365001733581), ('poor', 0.045802667097527436), ('low', 0.038833194358717134), ('hear', 0.035364652069384966), ('clear', 0.035364652069384966), ('loud', 0.03504068576005251), ('end', 0.033522433344890544)]
The top keywords for topic 28 are:
[('bluetooth', 0.2112060676449804), ('best', 0.15018701814569912), ('ive', 0.10372415014790246), ('transmitters', 0.08962925915975561), ('electronics', 0.08962925915975561), ('listener', 0.08962925915975561), ('blueant', 0.08962925915975561

We can see that topic 1 is about headsets and topic 0 is about sound quality. Both topics are a good fit for the new review. Topic 2 is about the earpiece, which is similar to the headset. From this example, we can see that the BERTopic model made good predictions on the new document.

# Step 10: Save and Load Topic Models

In step 10, we will talk about how to save and load BERTopic models.

The trained BERTopic model and its settings can be saved using `.save`. UMAP and HDBSCAN are saved, but the documents and embeddings are not saved.

We can use `.load` to load the saved BERTopic model.

In [ ]:
# Save the topic model
topic_model.save("amz_review_topic_model")

# Load the topic model
my_model = BERTopic.load("amz_review_topic_model")

# Recommended Tutorials

- [GrabNGoInfo Machine Learning Tutorials Inventory](https://medium.com/grabngoinfo/grabngoinfo-machine-learning-tutorials-inventory-9b9d78ebdd67)
- [Hierarchical Topic Model for Airbnb Reviews](https://medium.com/p/hierarchical-topic-model-for-airbnb-reviews-f772eaa30434)
- [3 Ways for Multiple Time Series Forecasting Using Prophet in Python](https://medium.com/p/3-ways-for-multiple-time-series-forecasting-using-prophet-in-python-7a0709a117f9)
- [Time Series Anomaly Detection Using Prophet in Python](https://medium.com/grabngoinfo/time-series-anomaly-detection-using-prophet-in-python-877d2b7b14b4)
- [Time Series Causal Impact Analysis in Python](https://medium.com/grabngoinfo/time-series-causal-impact-analysis-in-python-63eacb1df5cc)
- [Hyperparameter Tuning For XGBoost](https://medium.com/p/hyperparameter-tuning-for-xgboost-91449869c57e)
- [Four Oversampling And Under-Sampling Methods For Imbalanced Classification Using Python](https://medium.com/p/four-oversampling-and-under-sampling-methods-for-imbalanced-classification-using-python-7304aedf9037)
- [Five Ways To Create Tables In Databricks](https://medium.com/grabngoinfo/five-ways-to-create-tables-in-databricks-cd3847cfc3aa)
- [Explainable S-Learner Uplift Model Using Python Package CausalML](https://medium.com/grabngoinfo/explainable-s-learner-uplift-model-using-python-package-causalml-a3c2bed3497c)
- [One-Class SVM For Anomaly Detection](https://medium.com/p/one-class-svm-for-anomaly-detection-6c97fdd6d8af)
- [Recommendation System: Item-Based Collaborative Filtering](https://medium.com/grabngoinfo/recommendation-system-item-based-collaborative-filtering-f5078504996a)
- [Hyperparameter Tuning for Time Series Causal Impact Analysis in Python](https://medium.com/grabngoinfo/hyperparameter-tuning-for-time-series-causal-impact-analysis-in-python-c8f7246c4d22)
- [Hyperparameter Tuning and Regularization for Time Series Model Using Prophet in Python](https://medium.com/grabngoinfo/hyperparameter-tuning-and-regularization-for-time-series-model-using-prophet-in-python-9791370a07dc)
- [Multivariate Time Series Forecasting with Seasonality and Holiday Effect Using Prophet in Python](https://medium.com/p/multivariate-time-series-forecasting-with-seasonality-and-holiday-effect-using-prophet-in-python-d5d4150eeb57)
- [LASSO (L1) Vs Ridge (L2) Vs Elastic Net Regularization For Classification Model](https://medium.com/towards-artificial-intelligence/lasso-l1-vs-ridge-l2-vs-elastic-net-regularization-for-classification-model-409c3d86f6e9)
- [S Learner Uplift Model for Individual Treatment Effect and Customer Segmentation in Python](https://medium.com/grabngoinfo/s-learner-uplift-model-for-individual-treatment-effect-and-customer-segmentation-in-python-9d410746e122)
- [How to Use R with Google Colab Notebook](https://medium.com/p/how-to-use-r-with-google-colab-notebook-610c3a2f0eab)

# References

* [BERTopic GitHub](https://github.com/MaartenGr/BERTopic)
* [Documentation on BERTopic algorithms](https://maartengr.github.io/BERTopic/algorithm/algorithm.html#visual-overview)
* [UMAP documentation](https://umap-learn.readthedocs.io/en/latest/parameters.html)